# RAG Hybrid Search - GPU Model Comparison

Dense + Sparse ハイブリッド検索のマルチモデル比較（GPU版）

**ランタイム → ランタイムのタイプを変更 → T4 GPU** を選択してから実行してください。

## 1. セットアップ

In [ ]:
!pip install -q pymupdf sentence-transformers chromadb torch numpy tqdm

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")
else:
    print("WARNING: GPUが検出されません。ランタイム→ランタイムのタイプを変更→T4 GPUを選択してください。")

## 2. PDFアップロード

`TMC4361A_datasheet_rev1.26_01.pdf` をアップロードしてください。

In [ ]:
from google.colab import files
import os

PDF_PATH = "TMC4361A_datasheet_rev1.26_01.pdf"

if not os.path.exists(PDF_PATH):
    print("PDFファイルをアップロードしてください...")
    uploaded = files.upload()
    if uploaded:
        PDF_PATH = list(uploaded.keys())[0]
        print(f"Uploaded: {PDF_PATH}")
else:
    print(f"PDF already exists: {PDF_PATH}")

## 3. モジュール定義

In [ ]:
import fitz  # PyMuPDF
import re
from typing import List, Dict, Tuple


class PDFProcessor:
    def __init__(self, chunk_size: int = 512, chunk_overlap: int = 50):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc[page_num]
            text += f"\n--- Page {page_num + 1} ---\n"
            text += page.get_text()
        doc.close()
        return text

    def clean_text(self, text: str) -> str:
        text = re.sub(r'\n+', '\n', text)
        text = re.sub(r' +', ' ', text)
        return text.strip()

    def split_into_chunks(self, text: str) -> List[Dict[str, str]]:
        chunks = []
        start = 0
        text_length = len(text)
        chunk_id = 0
        while start < text_length:
            end = start + self.chunk_size
            if end < text_length:
                next_newline = text.find('\n', end)
                if next_newline != -1 and next_newline - end < 100:
                    end = next_newline
            chunk_text = text[start:end].strip()
            if chunk_text:
                chunks.append({
                    'id': f'chunk_{chunk_id}',
                    'text': chunk_text,
                    'start_pos': start,
                    'end_pos': end
                })
                chunk_id += 1
            start = end - self.chunk_overlap
        return chunks

    def process_pdf(self, pdf_path: str) -> List[Dict[str, str]]:
        print(f"Processing PDF: {pdf_path}")
        raw_text = self.extract_text_from_pdf(pdf_path)
        print(f"Extracted {len(raw_text)} characters")
        cleaned_text = self.clean_text(raw_text)
        print(f"Cleaned text: {len(cleaned_text)} characters")
        chunks = self.split_into_chunks(cleaned_text)
        print(f"Created {len(chunks)} chunks")
        return chunks

In [ ]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict


class HybridRAGSystem:
    def __init__(self, collection_name: str = "rag_documents",
                 model_name: str = "all-MiniLM-L6-v2",
                 device: str = None):
        print(f"Initializing RAG system with model: {model_name}")

        # デバイス自動検出
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = device
        print(f"  Device: {device}")

        self.dense_model = SentenceTransformer(model_name, device=device)

        self.client = chromadb.Client(Settings(
            anonymized_telemetry=False,
            allow_reset=True
        ))
        try:
            self.client.delete_collection(collection_name)
        except:
            pass
        self.collection = self.client.create_collection(
            name=collection_name,
            metadata={"hnsw:space": "cosine"}
        )
        print(f"  Collection '{collection_name}' initialized")

    def compute_dense_embedding(self, text: str) -> List[float]:
        embedding = self.dense_model.encode(text, convert_to_numpy=True)
        return embedding.tolist()

    def compute_dense_embeddings_batch(self, texts: List[str], batch_size: int = 64) -> List[List[float]]:
        """バッチでDense embeddingを計算（GPU活用）"""
        embeddings = self.dense_model.encode(texts, batch_size=batch_size,
                                             show_progress_bar=True,
                                             convert_to_numpy=True)
        return embeddings.tolist()

    def compute_sparse_embedding(self, text: str) -> Dict[str, float]:
        tokens = text.lower().split()
        tf = defaultdict(int)
        for token in tokens:
            tf[token] += 1
        total_tokens = len(tokens)
        return {token: count / total_tokens for token, count in tf.items()}

    def add_documents(self, chunks: List[Dict[str, str]], batch_size: int = 64):
        """ドキュメントをバッチ処理でベクトルDBに追加（GPU最適化）"""
        print(f"Adding {len(chunks)} documents (batch_size={batch_size})...")

        texts = [chunk['text'] for chunk in chunks]
        embeddings = self.compute_dense_embeddings_batch(texts, batch_size=batch_size)

        ids = [chunk['id'] for chunk in chunks]
        documents = texts
        metadatas = [{
            'start_pos': chunk['start_pos'],
            'end_pos': chunk['end_pos'],
            'sparse_tokens': len(self.compute_sparse_embedding(chunk['text'])),
        } for chunk in chunks]

        # ChromaDBは大きなバッチを分割して追加
        chroma_batch = 500
        for i in range(0, len(ids), chroma_batch):
            end = min(i + chroma_batch, len(ids))
            self.collection.add(
                ids=ids[i:end],
                embeddings=embeddings[i:end],
                documents=documents[i:end],
                metadatas=metadatas[i:end]
            )

        print(f"Successfully added {len(chunks)} documents")

    @staticmethod
    def min_max_normalize(scores: List[float]) -> np.ndarray:
        scores = np.array(scores)
        if len(scores) == 0:
            return scores
        min_s, max_s = scores.min(), scores.max()
        if max_s - min_s == 0:
            return np.ones_like(scores)
        return (scores - min_s) / (max_s - min_s)

    def dense_search(self, query: str, top_k: int = 10) -> List[Tuple[str, float, str]]:
        query_embedding = self.compute_dense_embedding(query)
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        formatted = []
        if results['ids'] and len(results['ids']) > 0:
            for i in range(len(results['ids'][0])):
                score = 1 - results['distances'][0][i]
                formatted.append((results['ids'][0][i], score, results['documents'][0][i]))
        return formatted

    def sparse_search(self, query: str, documents: List[Dict], top_k: int = 10) -> List[Tuple[str, float]]:
        query_tokens = set(query.lower().split())
        scores = []
        for doc in documents:
            doc_token_set = set(doc['text'].lower().split())
            intersection = query_tokens & doc_token_set
            union = query_tokens | doc_token_set
            score = len(intersection) / len(union) if len(union) > 0 else 0.0
            scores.append((doc['id'], score))
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:top_k]

    def hybrid_search(self, query: str, alpha: float = 0.5, top_k: int = 10) -> List[Dict]:
        dense_results = self.dense_search(query, top_k=top_k * 2)
        all_docs_result = self.collection.get()
        all_docs = [
            {'id': all_docs_result['ids'][i], 'text': all_docs_result['documents'][i]}
            for i in range(len(all_docs_result['ids']))
        ]
        sparse_results = self.sparse_search(query, all_docs, top_k=top_k * 2)

        doc_scores = {}
        dense_ids = [r[0] for r in dense_results]
        dense_scores = [r[1] for r in dense_results]
        dense_normalized = self.min_max_normalize(dense_scores)
        for i, doc_id in enumerate(dense_ids):
            doc_scores[doc_id] = {'dense': dense_normalized[i], 'sparse': 0.0}

        sparse_ids = [r[0] for r in sparse_results]
        sparse_scores_raw = [r[1] for r in sparse_results]
        sparse_normalized = self.min_max_normalize(sparse_scores_raw)
        for i, doc_id in enumerate(sparse_ids):
            if doc_id not in doc_scores:
                doc_scores[doc_id] = {'dense': 0.0, 'sparse': sparse_normalized[i]}
            else:
                doc_scores[doc_id]['sparse'] = sparse_normalized[i]

        final_scores = []
        for doc_id, scores in doc_scores.items():
            combined = alpha * scores['dense'] + (1 - alpha) * scores['sparse']
            final_scores.append((doc_id, combined, scores['dense'], scores['sparse']))
        final_scores.sort(key=lambda x: x[1], reverse=True)

        results = []
        for doc_id, combined_score, dense_score, sparse_score in final_scores[:top_k]:
            doc_result = self.collection.get(ids=[doc_id])
            if doc_result['documents']:
                results.append({
                    'id': doc_id,
                    'score': combined_score,
                    'dense_score': dense_score,
                    'sparse_score': sparse_score,
                    'text': doc_result['documents'][0]
                })
        return results

In [ ]:
TEST_CASES = [
    # カテゴリ1: 具体的数値検索 (Factual/Numeric)
    {"query": "What is the supply voltage range for TMC4361A?",
     "category": "factual_numeric", "difficulty": "easy",
     "expected_keywords": ["voltage", "VCC", "3.3", "5", "supply"]},
    {"query": "What is the maximum SPI clock frequency?",
     "category": "factual_numeric", "difficulty": "medium",
     "expected_keywords": ["SPI", "clock", "frequency", "MHz"]},
    {"query": "Operating temperature range",
     "category": "factual_numeric", "difficulty": "easy",
     "expected_keywords": ["temperature", "\u00b0C", "-40", "125", "operating"]},
    # カテゴリ2: 概念的質問 (Conceptual)
    {"query": "How does the S-shaped ramp generator work?",
     "category": "conceptual", "difficulty": "medium",
     "expected_keywords": ["ramp", "velocity", "acceleration", "profile", "S-shaped"]},
    {"query": "What is the purpose of closed-loop operation?",
     "category": "conceptual", "difficulty": "medium",
     "expected_keywords": ["closed-loop", "encoder", "feedback", "position", "control"]},
    {"query": "How does the encoder interface function?",
     "category": "conceptual", "difficulty": "medium",
     "expected_keywords": ["encoder", "interface", "incremental", "position", "ABN"]},
    # カテゴリ3: 専門用語・略語 (Technical Terms)
    {"query": "XTARGET register description",
     "category": "technical_terms", "difficulty": "easy",
     "expected_keywords": ["XTARGET", "register", "target", "position"]},
    {"query": "ChopSync feature explanation",
     "category": "technical_terms", "difficulty": "hard",
     "expected_keywords": ["ChopSync", "chopper", "synchronization"]},
    {"query": "SixPoint ramp mode",
     "category": "technical_terms", "difficulty": "medium",
     "expected_keywords": ["SixPoint", "ramp", "motion", "profile"]},
    # カテゴリ4: 複合条件検索 (Multi-aspect)
    {"query": "SPI communication protocol for motor speed control",
     "category": "multi_aspect", "difficulty": "hard",
     "expected_keywords": ["SPI", "speed", "control", "register", "velocity"]},
    {"query": "Encoder feedback for position tracking accuracy",
     "category": "multi_aspect", "difficulty": "hard",
     "expected_keywords": ["encoder", "position", "feedback", "accuracy", "deviation"]},
    {"query": "Reference switch configuration and homing procedure",
     "category": "multi_aspect", "difficulty": "hard",
     "expected_keywords": ["reference", "switch", "homing", "configuration"]},
    # カテゴリ5: 意味的類似検索 (Semantic Similarity)
    {"query": "How to make the motor move smoothly without vibration",
     "category": "semantic_similarity", "difficulty": "hard",
     "expected_keywords": ["ramp", "velocity", "smooth", "jerk", "acceleration"]},
    {"query": "Preventing the motor from losing track of its location",
     "category": "semantic_similarity", "difficulty": "hard",
     "expected_keywords": ["position", "encoder", "closed-loop", "stall", "deviation"]},
    {"query": "Connecting the chip to a microcontroller",
     "category": "semantic_similarity", "difficulty": "medium",
     "expected_keywords": ["SPI", "interface", "communication", "microcontroller", "connection"]},
]

## 4. モデル定義

CPU版の6モデル + GPU向け大型モデルを追加

In [ ]:
MODELS = [
    # --- CPU版と同じ6モデル（GPU上での速度比較用） ---
    {"name": "all-MiniLM-L6-v2",
     "dim": 384, "params": "22M", "type": "small",
     "description": "軽量ベースライン", "prefix": None},
    {"name": "all-mpnet-base-v2",
     "dim": 768, "params": "110M", "type": "base",
     "description": "SentenceTransformers最高品質", "prefix": None},
    {"name": "BAAI/bge-small-en-v1.5",
     "dim": 384, "params": "33M", "type": "small",
     "description": "BAAI小型", "prefix": "Represent this sentence: "},
    {"name": "intfloat/e5-small-v2",
     "dim": 384, "params": "33M", "type": "small",
     "description": "Microsoft E5小型", "prefix": "query: "},
    {"name": "thenlper/gte-small",
     "dim": 384, "params": "33M", "type": "small",
     "description": "Alibaba GTE小型", "prefix": None},

    # --- GPU向け大型モデル（CPUでは遅すぎたモデル） ---
    {"name": "BAAI/bge-base-en-v1.5",
     "dim": 768, "params": "110M", "type": "base",
     "description": "BAAI base。MTEBトップクラス", "prefix": "Represent this sentence: "},
    {"name": "BAAI/bge-large-en-v1.5",
     "dim": 1024, "params": "335M", "type": "large",
     "description": "BAAI large。最高精度クラス", "prefix": "Represent this sentence: "},
    {"name": "intfloat/e5-base-v2",
     "dim": 768, "params": "110M", "type": "base",
     "description": "Microsoft E5 base", "prefix": "query: "},
    {"name": "intfloat/e5-large-v2",
     "dim": 1024, "params": "335M", "type": "large",
     "description": "Microsoft E5 large", "prefix": "query: "},
    {"name": "thenlper/gte-base",
     "dim": 768, "params": "110M", "type": "base",
     "description": "Alibaba GTE base", "prefix": None},
    {"name": "thenlper/gte-large",
     "dim": 1024, "params": "335M", "type": "large",
     "description": "Alibaba GTE large", "prefix": None},
]

ALPHA_VALUES = [0.0, 0.5, 0.7, 1.0]

print(f"評価モデル数: {len(MODELS)}")
for m in MODELS:
    print(f"  {m['name']:<35} {m['params']:>5}  {m['dim']:>5}d  {m['description']}")

## 5. PDF処理

In [ ]:
processor = PDFProcessor(chunk_size=512, chunk_overlap=50)
chunks = processor.process_pdf(PDF_PATH)
print(f"\nChunk examples:")
for c in chunks[:2]:
    print(f"  [{c['id']}] {c['text'][:80]}...")

## 6. 評価実行

In [ ]:
import time
import json
import traceback


def evaluate_single_model(model_info, chunks, device="cuda"):
    model_name = model_info["name"]
    prefix = model_info.get("prefix")

    print(f"\n{'='*70}")
    print(f"Evaluating: {model_name} ({model_info['params']}, {model_info['dim']}d)")
    print(f"{'='*70}")

    # 初期化
    init_start = time.time()
    try:
        rag = HybridRAGSystem(
            collection_name=f"eval_{model_name.replace('/', '_').replace('-','_')}",
            model_name=model_name,
            device=device,
        )
    except Exception as e:
        print(f"  ERROR: {e}")
        return None
    init_time = time.time() - init_start

    # インデキシング
    add_start = time.time()
    rag.add_documents(chunks, batch_size=128)
    add_time = time.time() - add_start

    print(f"  Init: {init_time:.2f}s | Indexing: {add_time:.2f}s")

    # GPU メモリ
    gpu_mem_mb = 0
    if torch.cuda.is_available():
        gpu_mem_mb = torch.cuda.max_memory_allocated() / 1024**2

    model_results = {
        "model_name": model_name,
        "dim": model_info["dim"],
        "params": model_info["params"],
        "type": model_info["type"],
        "description": model_info["description"],
        "device": device,
        "init_time": round(init_time, 2),
        "indexing_time": round(add_time, 2),
        "gpu_memory_mb": round(gpu_mem_mb, 1),
        "alpha_results": {},
    }

    for alpha in ALPHA_VALUES:
        query_results = []
        for test in TEST_CASES:
            query = test["query"]
            q_for_dense = (prefix + query) if prefix else query

            search_start = time.time()
            if alpha == 1.0:
                raw = rag.dense_search(q_for_dense, top_k=5)
                formatted = [
                    {"id": r[0], "score": r[1], "text": r[2],
                     "dense_score": r[1], "sparse_score": 0.0}
                    for r in raw
                ]
            elif alpha == 0.0:
                formatted = rag.hybrid_search(query, alpha=0.0, top_k=5)
            else:
                formatted = rag.hybrid_search(q_for_dense, alpha=alpha, top_k=5)
            search_time = time.time() - search_start

            keyword_matches = []
            for result in formatted[:3]:
                text_lower = result["text"].lower()
                matches = sum(1 for kw in test["expected_keywords"] if kw.lower() in text_lower)
                keyword_matches.append(matches / len(test["expected_keywords"]))
            avg_kw = sum(keyword_matches) / len(keyword_matches) if keyword_matches else 0
            top_score = formatted[0]["score"] if formatted else 0

            query_results.append({
                "query": test["query"],
                "category": test["category"],
                "difficulty": test["difficulty"],
                "search_time": round(search_time, 4),
                "keyword_match_score": round(avg_kw, 4),
                "top_score": round(top_score, 4),
                "top_text_preview": formatted[0]["text"][:100] if formatted else "",
            })

        avg_st = np.mean([r["search_time"] for r in query_results])
        avg_kw = np.mean([r["keyword_match_score"] for r in query_results])
        avg_ts = np.mean([r["top_score"] for r in query_results])

        cat_scores = defaultdict(list)
        for r in query_results:
            cat_scores[r["category"]].append(r["keyword_match_score"])
        cat_avg = {c: round(np.mean(s), 4) for c, s in cat_scores.items()}

        model_results["alpha_results"][f"alpha_{alpha}"] = {
            "avg_search_time": round(avg_st, 4),
            "avg_keyword_match": round(avg_kw, 4),
            "avg_top_score": round(avg_ts, 4),
            "category_scores": cat_avg,
            "query_details": query_results,
        }

        print(f"  alpha={alpha}: KW={avg_kw:.4f} TopScore={avg_ts:.4f} Time={avg_st:.4f}s")

    # GPU メモリ解放
    del rag
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    return model_results

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(f"Models: {len(MODELS)} | Alpha values: {ALPHA_VALUES} | Test cases: {len(TEST_CASES)}")
print(f"Total evaluations: {len(MODELS) * len(ALPHA_VALUES) * len(TEST_CASES)}\n")

all_results = []
failed = []
total_start = time.time()

for i, mi in enumerate(MODELS):
    print(f"\n[{i+1}/{len(MODELS)}] {mi['name']}")
    try:
        r = evaluate_single_model(mi, chunks, device=device)
        if r:
            all_results.append(r)
        else:
            failed.append(mi["name"])
    except Exception as e:
        print(f"  FAILED: {e}")
        traceback.print_exc()
        failed.append(mi["name"])

total_time = time.time() - total_start
print(f"\n{'='*70}")
print(f"DONE: {len(all_results)} succeeded, {len(failed)} failed in {total_time:.1f}s")
if failed:
    print(f"Failed: {failed}")

## 7. 結果サマリー

In [ ]:
# alpha=0.7 でのランキング
print(f"{'='*90}")
print(f"RANKING (alpha=0.7) - sorted by Keyword Match Score")
print(f"{'='*90}")
print(f"{'#':<3} {'Model':<35} {'Params':>6} {'Dim':>5} {'KW Match':>9} {'TopScore':>9} {'Index(s)':>9} {'Search(s)':>10}")
print(f"{'-'*90}")

ranked = sorted(all_results,
                key=lambda x: x["alpha_results"].get("alpha_0.7", {}).get("avg_keyword_match", 0),
                reverse=True)

for rank, r in enumerate(ranked, 1):
    a = r["alpha_results"].get("alpha_0.7", {})
    print(f"{rank:<3} {r['model_name']:<35} {r['params']:>6} {r['dim']:>5} "
          f"{a.get('avg_keyword_match',0):>9.4f} {a.get('avg_top_score',0):>9.4f} "
          f"{r['indexing_time']:>9.2f} {a.get('avg_search_time',0):>10.4f}")

In [ ]:
# カテゴリ別ベスト (alpha=0.7)
print(f"\n{'='*70}")
print("BEST MODEL BY CATEGORY (alpha=0.7)")
print(f"{'='*70}")

cats = set()
for r in all_results:
    cats.update(r["alpha_results"].get("alpha_0.7", {}).get("category_scores", {}).keys())

for cat in sorted(cats):
    best_name, best_score = "", -1
    for r in all_results:
        s = r["alpha_results"].get("alpha_0.7", {}).get("category_scores", {}).get(cat, 0)
        if s > best_score:
            best_score = s
            best_name = r["model_name"]
    print(f"  {cat:<25} -> {best_name:<35} ({best_score:.4f})")

In [ ]:
# alpha別の最良モデル
print(f"\n{'='*70}")
print("BEST MODEL BY ALPHA VALUE")
print(f"{'='*70}")

for alpha in ALPHA_VALUES:
    key = f"alpha_{alpha}"
    best_name, best_score = "", -1
    for r in all_results:
        s = r["alpha_results"].get(key, {}).get("avg_keyword_match", 0)
        if s > best_score:
            best_score = s
            best_name = r["model_name"]
    print(f"  alpha={alpha}: {best_name:<35} (KW Match: {best_score:.4f})")

In [ ]:
# サイズ別比較（small vs base vs large）
print(f"\n{'='*70}")
print("SIZE CLASS COMPARISON (alpha=0.7)")
print(f"{'='*70}")

for size_class in ["small", "base", "large"]:
    models_in_class = [r for r in all_results if r["type"] == size_class]
    if not models_in_class:
        continue
    print(f"\n  [{size_class.upper()}]")
    for r in sorted(models_in_class,
                    key=lambda x: x["alpha_results"].get("alpha_0.7", {}).get("avg_keyword_match", 0),
                    reverse=True):
        a = r["alpha_results"].get("alpha_0.7", {})
        print(f"    {r['model_name']:<35} KW={a.get('avg_keyword_match',0):.4f}  "
              f"Index={r['indexing_time']:.1f}s  GPU={r.get('gpu_memory_mb',0):.0f}MB")

## 8. 結果保存 & ダウンロード

In [ ]:
output = {
    "evaluation_date": time.strftime("%Y-%m-%d %H:%M:%S"),
    "device": device,
    "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A",
    "num_chunks": len(chunks),
    "num_test_cases": len(TEST_CASES),
    "alpha_values": ALPHA_VALUES,
    "total_time_sec": round(total_time, 1),
    "failed_models": failed,
    "results": all_results,
}

output_path = "gpu_model_comparison_results.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

print(f"Results saved: {output_path}")
print(f"Total time: {total_time:.1f}s")

# ダウンロード
files.download(output_path)

## 9. CPU vs GPU 速度比較（オプション）

ベースラインモデルでCPU/GPU両方測定し、GPU高速化の効果を確認

In [ ]:
# CPU vs GPU 速度比較（all-MiniLM-L6-v2 のみ）
if torch.cuda.is_available():
    test_model = {"name": "all-MiniLM-L6-v2", "dim": 384, "params": "22M",
                  "type": "small", "description": "baseline", "prefix": None}

    print("CPU vs GPU Speed Comparison (all-MiniLM-L6-v2)")
    print("="*60)

    for dev in ["cpu", "cuda"]:
        r = evaluate_single_model(test_model, chunks, device=dev)
        if r:
            a07 = r["alpha_results"].get("alpha_0.7", {})
            print(f"\n  {dev.upper()}: Init={r['init_time']:.1f}s  "
                  f"Index={r['indexing_time']:.1f}s  "
                  f"Search={a07.get('avg_search_time',0):.4f}s/query")
else:
    print("GPU not available - skipping comparison")